# All Saxophones, Brass OK: SVM

* model = SVM
* features = MFCC
* target = ALL SAXES

Use SVM model to classify whether or not a saxophone is playing in a sample, samples including brass instruments are left in the dataset.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import AUC, Recall
from sklearn.model_selection import train_test_split, RandomizedSearchCV, \
    GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_curve, \
    roc_curve, roc_auc_score, confusion_matrix, \
    classification_report
from scipy.stats import uniform

from _common import NUM_LABEL_COLS, RANDOM_SEED
from _all_saxes_brass_ok_mfcc import TEST_SIZE, VALIDATION_SIZE, \
    BANDS, TIME_SLICES, data, target

In [2]:
NUM_COMPONENTS = 24

In [3]:
pca = PCA(n_components=NUM_COMPONENTS)    # optimal n_components value determined in mt_cc_all_saxes_brass_ok_pca_lmsc

In [4]:
pca.fit(data)

PCA(n_components=24)

In [5]:
# print(pca.explained_variance_ratio_)

In [6]:
# print(pca.singular_values_)

In [7]:
d = pca.transform(data)

In [8]:
# d

In [9]:
x_train, x_test, y_train, y_test = \
     train_test_split(d, target,
                      test_size=TEST_SIZE,
                      random_state=RANDOM_SEED)

## Test plain vanilla SVM

In [10]:
clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8088813940415964


In [11]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,553,203
1,137,886


In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.73      0.76       756
           1       0.81      0.87      0.84      1023

    accuracy                           0.81      1779
   macro avg       0.81      0.80      0.80      1779
weighted avg       0.81      0.81      0.81      1779



## Determine good value for C using random search

In [13]:
model_params = {
    'C': uniform(0.1, 3.0)
}

svm_model = SVC()
clf = RandomizedSearchCV(svm_model, model_params, n_iter=10, 
                         cv=5)
model = clf.fit(x_train, y_train)
params = model.best_estimator_.get_params()

In [14]:
clf = SVC(C=params['C'])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))


0.8251826869027543


In [15]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,585,171
1,140,883


In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.77      0.79       756
           1       0.84      0.86      0.85      1023

    accuracy                           0.83      1779
   macro avg       0.82      0.82      0.82      1779
weighted avg       0.82      0.83      0.82      1779



## Test ensemble approach with bagging classifier

In [20]:
# param_grid = {'max_features': [NUM_COMPONENTS - 2, NUM_COMPONENTS - 3, 
#                                NUM_COMPONENTS - 4, NUM_COMPONENTS - 5,
#                                NUM_COMPONENTS - 6, NUM_COMPONENTS - 7, 
#                                NUM_COMPONENTS - 8, NUM_COMPONENTS - 9,
#                                NUM_COMPONENTS - 10, NUM_COMPONENTS - 11,
#                                NUM_COMPONENTS - 12, NUM_COMPONENTS - 13]}

# base_estimator = SVC(C=params['C'])
# estimator = BaggingClassifier(base_estimator=base_estimator, n_estimators=100)

# clf = GridSearchCV(estimator, param_grid, n_jobs=-1)
# search = clf.fit(x_train, y_train)
# search.best_params_

# max_features = 17

{'max_features': 17}

In [21]:
clf = BaggingClassifier(base_estimator=SVC(C=params['C']),
                        n_estimators=400, 
                        max_features=17,
                        random_state=42)
model = clf.fit(x_train, y_train)

In [22]:
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8240584598088814


In [23]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,564,192
1,121,902


In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.75      0.78       756
           1       0.82      0.88      0.85      1023

    accuracy                           0.82      1779
   macro avg       0.82      0.81      0.82      1779
weighted avg       0.82      0.82      0.82      1779

